# Itinerary Generation

The purpose of this notebook is to generate the course site's itinerary page and subpages based on `Schedule.ipynb`

In [91]:
from datetime import date, timedelta
from helpers import calendar_table

In [92]:
SITE_DIR="../../../site"

In [93]:
%reload_ext autoreload
%run Schedule.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Date,Weekday,Plan
Week 0
Aug-18,Mon,FDOC
Aug-19,Tue,Day 00
Aug-21,Thu,Day 01
Week 1
Aug-26,Tue,Day 02
Aug-28,Thu,Day 03
Week 2
Sep-01,Mon,University Holiday
Sep-02,Tue,Day 04
Sep-04,Thu,Day 05
Week 3
Sep-09,Tue,Day 06
Sep-11,Thu,Day 07
Week 4
Sep-15,Mon,University Holiday
Sep-16,Tue,Day 08
Sep-18,Thu,Day 09
Week 5
Sep-23,Tue,Day 10
Sep-25,Thu,Day 11
Week 6
Sep-30,Tue,Day 12
Oct-02,Thu,Day 13
Week 7
Oct-07,Tue,University Holiday
Oct-09,Thu,Day 14
Week 8
Oct-14,Tue,Day 15
Oct-16,Thu,University Holiday
Oct-17,Fri,University Holiday
Week 9
Oct-21,Tue,Day 16
Oct-23,Thu,Day 17
Oct-25,Sat,Drop Period Ends
Week 10
Oct-28,Tue,Day 18
Oct-30,Thu,Day 19
Week 11
Nov-04,Tue,Day 20
Nov-06,Thu,Day 21
Week 12
Nov-11,Tue,Day 22
Nov-13,Thu,Day 23
Week 13
Nov-17,Mon,Widthdraw Period Ends
Nov-18,Tue,Day 24
Nov-20,Thu,Day 25
Week 14
Nov-25,Tue,Day 26
Nov-26,Wed,University Holiday
Nov-27,Thu,University Holiday
Nov-28,Fri,

We're going to curate what is in the full syllabus based on what we want to expose to students.

In [94]:
today = date.today() - timedelta(2)  # Hack to keep the past three days in range for now, need to rewrite for deadlines

def keep(plan):
    # Don't show "Class" or "Milestone"
    if plan.kind == Kind.MS or plan.kind == Kind.DK or plan.kind == Kind.NA:
        return False
    elif plan.kind == Kind.WD and plan.date - today > timedelta(6):
        # Filter holidays out in the far future, but keep nearby ones in for
        # positive vibes and unrelenting optimism.
        return False
    elif (plan.kind == Kind.CL or plan.kind == Kind.RV) and len(plan.links) == 0:
        return False
    elif plan.kind != Kind.QZ and plan.kind != Kind.EX and plan.kind != Kind.RD and plan.kind != Kind.FN and plan.date - today > timedelta(6):
        return False
    else:
        return True

for day in syllabus:
    remove = []
    syllabus[day] = [plan for plan in syllabus[day] if keep(plan)]

show_calendar(syllabus)

Date,Weekday,Plan
Week 0,,
Aug-19,Tue,Welcome to COMP110
,,Out: Syllabus
,,Out: Hello World
,,Due: Syllabus
Aug-22,Fri,Due: Hello World
Week 1,,
Aug-28,Thu,Out: Quiz 0
,,Due: Quiz 0
Week 2,,


In [95]:
# Segment out future vs past
future = {}
past = {}
for day in syllabus:
    if day < today:
        past[day] = syllabus[day]
    else:
        future[day] = syllabus[day]

Similar to itinerary generation, we will auto-generate the 'On the Horizon' box. First, let's curate what is in the box:

In [96]:
from helpers import horizonize

horizon_syl = horizonize(syllabus_copy)
horizon_syl = [plan for plan in horizon_syl if plan.kind in {Kind.EX, Kind.QZ, Kind.RD, Kind.HW, Kind.FN, Kind.LS, Kind.PJ, Kind.CQ, Kind.VL}]

Stripping data to only near future quizzes and due dates

In [97]:
horizoninfo = []
today = date.today()

for plan in horizon_syl:
    if plan.date > (today - timedelta(1)) and plan.date < (today + timedelta(8)):
        horizoninfo.append(plan)

print(horizoninfo)

[<helpers.HorizonPlan object at 0x10910ed90>, <helpers.HorizonPlan object at 0x109160750>]


Working to add deadline in titles of assignments where assignment date != due date

In [98]:
import calendar
syllabus: dict[date, list[Plan]]
for day in syllabus:
    for plan in syllabus[day]: 
        if plan.deadline != plan.extension:
            newdate = plan.extension.strftime("%-m/%-d")
            date = plan.deadline.strftime("%-m/%-d")
            plan.title += f"<s><i>(Due {calendar.day_name[plan.deadline.weekday()]} {date})</i></s>" 
            plan.title += f" (Extended to {calendar.day_name[plan.extension.weekday()]} {newdate})" 
        elif (plan.deadline != None) and (plan.kind != Kind.QZ): 
            date = plan.deadline.strftime("%-m/%-d")
            plan.title += f" (Due {calendar.day_name[plan.deadline.weekday()]} {date})" 

And now (finally), we generate the itinerary and horizon box.

In [99]:
from jinja2 import Environment, FileSystemLoader, select_autoescape
from helpers import serialize, serialize_plan
import re
import json 
from datetime import date
env = Environment(
    loader=FileSystemLoader(SITE_DIR),
    autoescape=select_autoescape(["html", "xml"]),
    trim_blocks=True,
    lstrip_blocks=True
)
template = env.get_template("./index.jinja2")
with open(f"{SITE_DIR}/index.md", "w", encoding="utf8") as target:
    result = template.render(syllabus=syllabus, horizon_syl=horizon_syl, future=future, horizoninfo=horizoninfo, past=past, cache=date.today())
    # We strip the double spaces because pandoc can't handle HTML well :(
    stripped = re.sub(r' {2,}', '', result)
    target.write(stripped)
serializable_syllabus = serialize(syllabus) 
future_syllabus = serialize(future)  
past_syllabus = serialize(past)   
json_syllabus = json.dumps(serializable_syllabus, indent = 4)   
json_future = json.dumps(future_syllabus, indent = 4)
json_past = json.dumps(past_syllabus, indent = 4)
f = open(f"{SITE_DIR}/static/components/Itinerary/data.json", "w")
f.write(json_syllabus)
f.close()
f = open(f"{SITE_DIR}/static/components/Itinerary/future.json", "w")
f.write(json_future)
f.close()
f = open(f"{SITE_DIR}/static/components/Itinerary/past.json", "w")
f.write(json_past)
f.close()

In [100]:
# for slide in LS_links:
    # print(slide)